# Pre-Analysis Stuff


## General Information

Baseline includes 1,090 (a random number more than 1000) cases taken from the Orignal Manifesto. The cases were downloaded using the gdc client. Notice how the base line has an executable file called gdc clinet => it is `./gdc-client download -m updated.txt `

I used all the AML cases here since for V617F, 2 out of 4 cases were from AML, 1 from BetaAML and one was from liver. 

Also for all the Jak2 Mutationas, AML was the most common!


## How the files are structured

The files are structured as follows

1. **Baseline** includes 1090 AML cases that we use to form a baseline values for all the gene expression levels

2. **Deleterious** includes 32 cases that have a **high** Polyphen scored mutation in Jak2 - all of them are missense mutations. 

    a. The mutations are structured as follows - ![Image of all the deleterious mutations](polyphen.png)

    Here we use everything that has an impact of PR. Data collected from here - https://portal.gdc.cancer.gov/genes/ENSG00000096968

3. **V617F** includes all the cases that have the V617f mutations




## The Pre-Analysis work

The files are structured such that each folder has folders for all the case and than each case folder has a tab seperated value aka .tsv file that has data. 







In [1]:
#### Already used it no need to use this code again!

# import pandas as pd
# import random

# # Load the manifest file
# file_path = 'GDC Manifest Aug 22.txt'

# # Read the file into a DataFrame
# df = pd.read_csv(file_path, sep='\t')

# # Randomly select 1000 rows
# df_sampled = df.sample(n=1090, random_state=420)

# # Save the sampled DataFrame to a new file
# output_path = 'manifest.txt'
# df_sampled.to_csv(output_path, sep='\t', index=False)

# output_path


In [2]:
# import os
# import shutil

# # Define paths
# tcga_folder = 'TCGA'
# baseline_folder = os.path.join(tcga_folder, 'baseline')
# data_folder = os.path.join(baseline_folder, 'data')

# # Create the data folder if it doesn't exist
# os.makedirs(data_folder, exist_ok=True)

# # Determine the starting file_counter based on existing files in data folder
# existing_files = [f for f in os.listdir(data_folder) if f.endswith('.tsv')]
# if existing_files:
#     existing_numbers = [int(os.path.splitext(f)[0]) for f in existing_files]
#     file_counter = max(existing_numbers) + 1
# else:
#     file_counter = 1

# # Function to clean up folders, move files, and delete empty folders
# def clean_and_reorganize_folders(baseline_folder, data_folder):
#     global file_counter
#     for case_folder in os.listdir(baseline_folder):
#         case_path = os.path.join(baseline_folder, case_folder)
        
#         # Skip non-directory files like .DS_Store
#         if not os.path.isdir(case_path):
#             continue
        
#         log_folder = os.path.join(case_path, 'logs')
        
#         # Remove the logs directory and its contents recursively
#         if os.path.exists(log_folder):
#             shutil.rmtree(log_folder)
        
#         # Move the .tsv file to the data folder and rename it
#         for file_name in os.listdir(case_path):
#             if file_name.endswith('.tsv'):
#                 src_file_path = os.path.join(case_path, file_name)
#                 dest_file_path = os.path.join(data_folder, f"{file_counter}.tsv")
#                 shutil.move(src_file_path, dest_file_path)
#                 file_counter += 1
        
#         # Remove the now-empty case folder
#         if not os.listdir(case_path):  # Check if the directory is empty
#             os.rmdir(case_path)

# # Clean and reorganize baseline folders
# clean_and_reorganize_folders(baseline_folder, data_folder)

# print(f"Data files have been moved to '{data_folder}' and renamed sequentially. Empty folders have been deleted.")


In [3]:
# import os

# # Define the path to the data folder
# data_folder = os.path.join('TCGA', 'baseline', 'data')

# # Get a list of all .tsv files in the data folder
# files = [f for f in os.listdir(data_folder) if f.endswith('.tsv')]

# # Sort the files to ensure consistent renaming
# files.sort()

# # Rename each file sequentially from 1.tsv onwards
# for i, file_name in enumerate(files, start=1):
#     old_file_path = os.path.join(data_folder, file_name)
#     new_file_name = f"{i}.tsv"
#     new_file_path = os.path.join(data_folder, new_file_name)
#     os.rename(old_file_path, new_file_path)

# print(f"Renamed {len(files)} files sequentially from 1.tsv to {len(files)}.tsv.")


In [4]:
# import os
# import shutil

# # Define the paths
# tcga_folder = 'TCGA'
# baseline_folder = os.path.join(tcga_folder, 'baseline')
# data_folder = os.path.join(baseline_folder, 'data')

# # Function to clean up all folders in TCGA except for the 'data' folder
# def clean_tcga_folder(tcga_folder, data_folder):
#     for root, dirs, files in os.walk(baseline_folder, topdown=False):
#         for name in dirs:
#             folder_path = os.path.join(root, name)
#             if folder_path != data_folder:
#                 shutil.rmtree(folder_path)
#                 print(f"Deleted folder: {folder_path}")

# # Clean up the folders
# clean_tcga_folder(tcga_folder, data_folder)

# print(f"Cleaned up all folders in '{tcga_folder}' except for the 'data' folder.")


# Cleaning

In [5]:
import os
import numpy as np
import pandas as pd
from tqdm import tqdm

# Define paths
data_folder = os.path.join('TCGA', 'baseline', 'data')
output_trimmed_baseline_file = os.path.join('TCGA', 'baseline', 'trimmed_baseline_data.tsv')

# Initialize lists to hold the gene IDs and expression data
gene_ids = []
gene_names = []
gene_types = []
expression_data = None

# Get the list of all .tsv files
tsv_files = sorted([f for f in os.listdir(data_folder) if f.endswith('.tsv')])

# Iterate through all the TSV files and collect data
for i, file_name in enumerate(tqdm(tsv_files, desc="Processing files")):
    file_path = os.path.join(data_folder, file_name)
    df = pd.read_csv(file_path, sep='\t', comment='#')
    
    if i == 0:
        gene_ids = df['gene_id'].values
        gene_names = df['gene_name'].values
        gene_types = df['gene_type'].values
        expression_data = np.zeros((len(gene_ids), len(tsv_files)), dtype=np.float32)
    
    # Use numpy array operations to fill in the data matrix
    expression_data[:, i] = df['unstranded'].values

# Calculate the trimmed mean across all cases
trimmed_mean_expression = np.zeros(len(gene_ids), dtype=np.float32)

for j in range(expression_data.shape[0]):
    # Sort the values for the gene and trim the top and bottom 1%
    sorted_values = np.sort(expression_data[j, :])
    trimmed_values = sorted_values[int(0.01 * len(sorted_values)) : int(0.99 * len(sorted_values))]
    trimmed_mean_expression[j] = np.mean(trimmed_values)

# Calculate the median across all cases (unchanged)
median_expression = np.median(expression_data, axis=1)

# Create a new DataFrame to hold the trimmed baseline data
trimmed_baseline_df = pd.DataFrame({
    'gene_id': gene_ids,
    'gene_name': gene_names,
    'gene_type': gene_types,
    'trimmed_mean_expression': trimmed_mean_expression,
    'median_expression': median_expression
})

# Save the trimmed baseline data to a new TSV file
trimmed_baseline_df.to_csv(output_trimmed_baseline_file, sep='\t', index=False)

print(f"Trimmed baseline data saved to '{output_trimmed_baseline_file}'")


Processing files: 100%|██████████| 1036/1036 [00:41<00:00, 24.73it/s]


Trimmed baseline data saved to 'TCGA/baseline/trimmed_baseline_data.tsv'
